In [14]:

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    """
    Generates a bill for the given month based on the item list.

    Parameters:
    item_list (list): List of dictionaries with item details.
    target_month (str): Month in "YYYY-MM" format (e.g., "2024-11").

    Returns:
    dict: A dictionary with grouped line items and total revenue.
    """

    def days_in_month(year, month):
        if month == 2:
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                return 29
            else:
                return 28
        elif month in [4, 6, 9, 11]:
            return 30
        else:
            return 31

    def date_to_days(date_str):
        year, month, day = map(int, date_str.split('-'))
        days = 0
        for y in range(1970, year):
            days += 366 if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0) else 365
        for m in range(1, month):
            days += days_in_month(year, m)
        days += day
        return days

    def get_month_range_days(year_month):
        year, month = map(int, year_month.split('-'))
        start_day = date_to_days(f"{year}-{month}-01")
        end_day = date_to_days(f"{year}-{month}-{days_in_month(year, month)}")
        return start_day, end_day

    line_items = []
    total_revenue = 0.0
    grouped_items = {}

    target_month_start_days, target_month_end_days = get_month_range_days(target_month)
    target_year, target_month_num = map(int, target_month.split('-'))
    days_in_target_month = days_in_month(target_year, target_month_num)

    for item in item_list:
        try:
            item_start_date_str = str(item.get("start_date", ""))
            item_stop_date_str = str(item.get("stop_date", ""))

            if not item_start_date_str or not item_stop_date_str:
                continue

            item_start_days = date_to_days(item_start_date_str)
            item_stop_days = date_to_days(item_stop_date_str)

            
            if not (item_stop_days < target_month_start_days or item_start_days > target_month_end_days):
                # Item is active in the target month
                active_start_days = max(item_start_days, target_month_start_days)
                active_end_days = min(item_stop_days, target_month_end_days)
                active_days = active_end_days - active_start_days + 1

                rate = float(item.get("rate", 0))
                qty = int(item.get("qty", 0))

                
                amount = (rate / days_in_target_month) * active_days * qty

                
                def days_to_date_str(days):
                    year = 1970
                    while True:
                        days_in_year = 366 if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0) else 365
                        if days <= days_in_year:
                            break
                        days -= days_in_year
                        year += 1
                    month = 1
                    while True:
                        days_in_m = days_in_month(year, month)
                        if days <= days_in_m:
                            break
                        days -= days_in_m
                        month += 1
                    return f"{year}-{month:02}-{days:02}"

                billing_period_start_days = max(item_start_days, target_month_start_days)
                billing_period_end_days = min(item_stop_days, target_month_end_days)
                billing_period = f"{days_to_date_str(billing_period_start_days)} to {days_to_date_str(billing_period_end_days)}"

                group_key = (item["item_code"], rate, billing_period)
                if group_key not in grouped_items:
                    grouped_items[group_key] = {
                        "item_code": item["item_code"],
                        "rate": rate,
                        "qty": qty,
                        "amount": amount,
                        "billing_period": billing_period
                    }
                    line_items.append(grouped_items[group_key])
                else:
                    grouped_items[group_key]["qty"] += qty
                    grouped_items[group_key]["amount"] += amount

                
                total_revenue += amount

        except (ValueError, TypeError, KeyError) as e:
            print(f"Error processing item: {item}. Error: {e}")

    return {
        "line_items": line_items,
        "total_revenue": total_revenue
    }

 
item_list = [
    {
        "idx": 1,
        "item_code": "Executive Desk (4*2)",
        "sales_description": "Dedicated Executive Desk",
        "qty": "10",
        "rate": "1000",
        "amount": "10000",
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 2,
        "item_code": "Executive Desk (4*2)",
        "qty": "10",
        "rate": "1080",
        "amount": "10800",
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 3,
        "item_code": "Executive Desk (4*2)",
        "qty": 15,
        "rate": "1080",
        "amount": "16200",
        "start date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 4,
        "item_code": "Executive Desk (4*2)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 5,
        "item_code": "Manager Cabin",
        "qty": 5,
        "rate": 5000,
        "amount": 25000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 6,
        "item_code": "Manager Cabin",
        "qty": 7,
        "rate": "5000",
        "amount": "35000",
        "start_date": "2024-12-15",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 7,
        "item_code": "Manager Cabin",
        "qty": 10,
        "rate": 4600,
        "amount": 46000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 8,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 9,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 0,
        "amount": 0,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 10,
        "item_code": "Executive Desk (4*2)",
        "qty": "8",
        "rate": "1100",
        "amount": "8800",
        "start_date": "2024-11-15",
        "stop_date": "2025-01-31",
    },
    {
        "idx": 11,
        "item_code": "Manager Cabin",
        "qty": "3",
        "rate": "5200",
        "amount": "15600",
        "start_date": "2024-10-10",
        "stop_date": "2024-11-10",
    },
    {
        "idx": 12,
        "item_code": "Conference Table",
        "qty": 1,
        "rate": "20000",
        "amount": "20000",
        "start_date": "2024-11-05",
        "stop_date": "2024-11-20",
    },
    {
        "idx": 13,
        "item_code": "Parking (2S)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-15",
        "stop_date": "2025-02-28",
    },
    {
        "idx": 14,
        "item_code": "Reception Desk",
        "qty": 2,
        "rate": "7000",
        "amount": "14000",
        "start_date": "2024-11-01",
        "stop_date": "2025-03-31",
    },
    {
        "idx": 15,
        "item_code": "Reception Desk",
        "qty": 1,
        "rate": "7000",
        "amount": "7000",
        "start_date": "2024-11-10",
        "stop_date": "2024-11-25",
    },
    {
         "idx": 16,
         "item_code": "Breakout Area",
         "qty": 3,
         "rate": "3000",
         "amount": "9000",
         "start_date": "2024-01-01",
         "stop_date": "2024-01-31",
     }
]

target_month = "2024-11"
bill = generate_monthly_bill(item_list, target_month)
print(bill)

{'line_items': [{'item_code': 'Executive Desk (4*2)', 'rate': 1080.0, 'qty': 10, 'amount': 10800.0, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Executive Desk (4*2)', 'rate': 1000.0, 'qty': 5, 'amount': 5000.000000000001, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Manager Cabin', 'rate': 5000.0, 'qty': 5, 'amount': 25000.0, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Parking (2S)', 'rate': 1000.0, 'qty': 10, 'amount': 10000.000000000002, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Parking (2S)', 'rate': 0.0, 'qty': 10, 'amount': 0.0, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Executive Desk (4*2)', 'rate': 1100.0, 'qty': 8, 'amount': 4693.333333333333, 'billing_period': '2024-11-15 to 2024-11-30'}, {'item_code': 'Manager Cabin', 'rate': 5200.0, 'qty': 3, 'amount': 5200.0, 'billing_period': '2024-11-01 to 2024-11-10'}, {'item_code': 'Conference Table', 'rate': 20000.0, 'qty': 1, 'amount': 10666.6

In [15]:

def generate_monthly_bill(item_list, target_month):
     
    def days_in_month(year, month):
        if month == 2:
            return 28
        elif month in [4, 6, 9, 11]:
            return 30
        else:
            return 31

    
    def parse_date(date_str):
        parts = date_str.split('-')
        return int(parts[0]), int(parts[1]), int(parts[2])

     
    def date_less_equal(d1, d2):
        return d1 <= d2

    def date_greater_equal(d1, d2):
        return d1 >= d2

     
    def get_overlap_days(item_start, item_end, month_start, month_end):
        overlap_start = max(item_start, month_start)
        overlap_end = min(item_end, month_end)
        if overlap_start > overlap_end:
            return 0, "", ""
        days = (date_to_day(overlap_end) - date_to_day(overlap_start)) + 1
        return days, date_to_str(overlap_start), date_to_str(overlap_end)

     
    def date_to_day(date_tuple):
        y, m, d = date_tuple
        total_days = 0
        for i in range(1, m):
            total_days += days_in_month(y, i)
        total_days += d
        return total_days

    def date_to_str(date_tuple):
        y, m, d = date_tuple
        return f"{y:04d}-{m:02d}-{d:02d}"

     
    year, month = map(int, target_month.split('-'))
    month_start = (year, month, 1)
    month_end = (year, month, days_in_month(year, month))
    total_days = days_in_month(year, month)

    grouped = {}

    for item in item_list:
        try:
            start_date = parse_date(item["start_date"])
            stop_date = parse_date(item["stop_date"])
        except:
            continue

        overlap_days, p_start, p_end = get_overlap_days(start_date, stop_date, month_start, month_end)
        if overlap_days == 0:
            continue

        qty = int(item["qty"])
        rate = float(item["rate"])
        amount = round(qty * rate * (overlap_days / total_days), 2)
        key = (item["item_code"], rate, f"{p_start} to {p_end}")

        if key not in grouped:
            grouped[key] = {"qty": 0, "amount": 0}
        grouped[key]["qty"] += qty
        grouped[key]["amount"] += amount

    line_items = []
    total_revenue = 0

    for key, values in grouped.items():
        item_code, rate, billing_period = key
        amt = round(values["amount"], 2)
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": values["qty"],
            "amount": amt,
            "billing_period": billing_period
        })
        total_revenue += amt

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }
     
items = [
    {
        "item_code": "CHAIR",
        "rate": "100",
        "qty": "2",
        "start_date": "2023-05-10",
        "stop_date": "2023-06-10"
    },
    {
        "item_code": "DESK",
        "rate": "250",
        "qty": "1",
        "start_date": "2023-05-01",
        "stop_date": "2023-05-31"
    },
    {
        "item_code": "CHAIR",
        "rate": "100",
        "qty": "1",
        "start_date": "2023-05-20",
        "stop_date": "2023-05-25"
    },
    {
        "item_code": "PROJECTOR",
        "rate": "500",
        "qty": "1",
        "start_date": "2023-04-15",
        "stop_date": "2023-04-30"
    }  
]

 
month = "2023-05"

 
bill = generate_monthly_bill(items, month)

 
print("Monthly Bill Summary:")
print(f"Total Revenue: ₹{bill['total_revenue']}\n")
print("Line Items:")
for item in bill["line_items"]:
    print(f"- Item Code: {item['item_code']}")
    print(f"  Qty: {item['qty']}")
    print(f"  Rate: ₹{item['rate']}")
    print(f"  Amount: ₹{item['amount']}")
    print(f"  Billing Period: {item['billing_period']}")
    print()




Monthly Bill Summary:
Total Revenue: ₹411.29

Line Items:
- Item Code: CHAIR
  Qty: 2
  Rate: ₹100.0
  Amount: ₹141.94
  Billing Period: 2023-05-10 to 2023-05-31

- Item Code: DESK
  Qty: 1
  Rate: ₹250.0
  Amount: ₹250.0
  Billing Period: 2023-05-01 to 2023-05-31

- Item Code: CHAIR
  Qty: 1
  Rate: ₹100.0
  Amount: ₹19.35
  Billing Period: 2023-05-20 to 2023-05-25



In [13]:

result = generate_monthly_bill(items, month)

print("{")
print('  "line_items": [')

for idx, item in enumerate(result["line_items"]):
    print("    {")
    print(f'      "item_code": "{item["item_code"]}",')
    print(f'      "rate": {item["rate"]},')
    print(f'      "qty": {item["qty"]},')
    print(f'      "amount": {item["amount"]},')
    print(f'      "billing_period": "{item["billing_period"]}"')
    end_comma = "," if idx < len(result["line_items"]) - 1 else ""
    print(f"    }}{end_comma}")

print("  ],")
print(f'  "total_revenue": {result["total_revenue"]}')
print("}")


{
  "line_items": [
    {
      "item_code": "CHAIR",
      "rate": 100.0,
      "qty": 2,
      "amount": 141.94,
      "billing_period": "2023-05-10 to 2023-05-31"
    },
    {
      "item_code": "DESK",
      "rate": 250.0,
      "qty": 1,
      "amount": 250.0,
      "billing_period": "2023-05-01 to 2023-05-31"
    },
    {
      "item_code": "CHAIR",
      "rate": 100.0,
      "qty": 1,
      "amount": 19.35,
      "billing_period": "2023-05-20 to 2023-05-25"
    }
  ],
  "total_revenue": 411.29
}


In [16]:

{
  "line_items": [
    {
      "item_code": "CHAIR",
      "rate": 100.0,
      "qty": 2,
      "amount": 141.94,
      "billing_period": "2023-05-10 to 2023-05-31"
    },
    {
      "item_code": "DESK",
      "rate": 250.0,
      "qty": 1,
      "amount": 250.0,
      "billing_period": "2023-05-01 to 2023-05-31"
    },
    {
      "item_code": "CHAIR",
      "rate": 100.0,
      "qty": 1,
      "amount": 22.58,
      "billing_period": "2023-05-20 to 2023-05-25"
    }
  ],
  "total_revenue": 414.52
}


{'line_items': [{'item_code': 'CHAIR',
   'rate': 100.0,
   'qty': 2,
   'amount': 141.94,
   'billing_period': '2023-05-10 to 2023-05-31'},
  {'item_code': 'DESK',
   'rate': 250.0,
   'qty': 1,
   'amount': 250.0,
   'billing_period': '2023-05-01 to 2023-05-31'},
  {'item_code': 'CHAIR',
   'rate': 100.0,
   'qty': 1,
   'amount': 22.58,
   'billing_period': '2023-05-20 to 2023-05-25'}],
 'total_revenue': 414.52}

In [17]:


def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    """
    Generates a bill for the given month based on the item list.

    Parameters:
    item_list (list): List of dictionaries with item details.
    target_month (str): Month in "YYYY-MM" format (e.g., "2024-11").

    Returns:
    dict: A dictionary with grouped line items and total revenue.
    """

    def days_in_month(year, month):
        if month == 2:
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                return 29
            else:
                return 28
        elif month in [4, 6, 9, 11]:
            return 30
        else:
            return 31

    def date_to_days(date_str):
        year, month, day = map(int, date_str.split('-'))
        days = 0
        for y in range(1970, year):
            days += 366 if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0) else 365
        for m in range(1, month):
            days += days_in_month(year, m)
        days += day
        return days

    def get_month_range_days(year_month):
        year, month = map(int, year_month.split('-'))
        start_day = date_to_days(f"{year}-{month}-01")
        end_day = date_to_days(f"{year}-{month}-{days_in_month(year, month)}")
        return start_day, end_day

    line_items = []
    total_revenue = 0.0
    grouped_items = {}

    target_month_start_days, target_month_end_days = get_month_range_days(target_month)
    target_year, target_month_num = map(int, target_month.split('-'))
    days_in_target_month = days_in_month(target_year, target_month_num)

    for item in item_list:
        try:
            item_start_date_str = str(item.get("start_date", ""))
            item_stop_date_str = str(item.get("stop_date", ""))

            if not item_start_date_str or not item_stop_date_str:
                continue

            item_start_days = date_to_days(item_start_date_str)
            item_stop_days = date_to_days(item_stop_date_str)

            # 1. Active Item Detection
            if not (item_stop_days < target_month_start_days or item_start_days > target_month_end_days):
                # Item is active in the target month
                active_start_days = max(item_start_days, target_month_start_days)
                active_end_days = min(item_stop_days, target_month_end_days)
                active_days = active_end_days - active_start_days + 1

                rate = float(item.get("rate", 0))
                qty = int(item.get("qty", 0))

                # 3. Amount Calculation
                amount = (rate / days_in_target_month) * active_days * qty

                # 2. Grouping Logic
                def days_to_date_str(days):
                    year = 1970
                    while True:
                        days_in_year = 366 if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0) else 365
                        if days <= days_in_year:
                            break
                        days -= days_in_year
                        year += 1
                    month = 1
                    while True:
                        days_in_m = days_in_month(year, month)
                        if days <= days_in_m:
                            break
                        days -= days_in_m
                        month += 1
                    return f"{year}-{month:02}-{days:02}"

                billing_period_start_days = max(item_start_days, target_month_start_days)
                billing_period_end_days = min(item_stop_days, target_month_end_days)
                billing_period = f"{days_to_date_str(billing_period_start_days)} to {days_to_date_str(billing_period_end_days)}"

                group_key = (item["item_code"], rate, billing_period)
                if group_key not in grouped_items:
                    grouped_items[group_key] = {
                        "item_code": item["item_code"],
                        "rate": rate,
                        "qty": qty,
                        "amount": amount,
                        "billing_period": billing_period
                    }
                    line_items.append(grouped_items[group_key])
                else:
                    grouped_items[group_key]["qty"] += qty
                    grouped_items[group_key]["amount"] += amount

                # 4. Total Revenue
                total_revenue += amount

        except (ValueError, TypeError, KeyError) as e:
            print(f"Error processing item: {item}. Error: {e}")

    return {
        "line_items": line_items,
        "total_revenue": total_revenue
    }

# --- Example Usage ---
item_list = [
    {
        "idx": 1,
        "item_code": "Executive Desk (4*2)",
        "sales_description": "Dedicated Executive Desk",
        "qty": "10",
        "rate": "1000",
        "amount": "10000",
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 2,
        "item_code": "Executive Desk (4*2)",
        "qty": "10",
        "rate": "1080",
        "amount": "10800",
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 3,
        "item_code": "Executive Desk (4*2)",
        "qty": 15,
        "rate": "1080",
        "amount": "16200",
        "start date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 4,
        "item_code": "Executive Desk (4*2)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 5,
        "item_code": "Manager Cabin",
        "qty": 5,
        "rate": 5000,
        "amount": 25000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 6,
        "item_code": "Manager Cabin",
        "qty": 7,
        "rate": "5000",
        "amount": "35000",
        "start_date": "2024-12-15",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 7,
        "item_code": "Manager Cabin",
        "qty": 10,
        "rate": 4600,
        "amount": 46000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 8,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 9,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 0,
        "amount": 0,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 10,
        "item_code": "Executive Desk (4*2)",
        "qty": "8",
        "rate": "1100",
        "amount": "8800",
        "start_date": "2024-11-15",
        "stop_date": "2025-01-31",
    },
    {
        "idx": 11,
        "item_code": "Manager Cabin",
        "qty": "3",
        "rate": "5200",
        "amount": "15600",
        "start_date": "2024-10-10",
        "stop_date": "2024-11-10",
    },
    {
        "idx": 12,
        "item_code": "Conference Table",
        "qty": 1,
        "rate": "20000",
        "amount": "20000",
        "start_date": "2024-11-05",
        "stop_date": "2024-11-20",
    },
    {
        "idx": 13,
        "item_code": "Parking (2S)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-15",
        "stop_date": "2025-02-28",
    },
    {
        "idx": 14,
        "item_code": "Reception Desk",
        "qty": 2,
        "rate": "7000",
        "amount": "14000",
        "start_date": "2024-11-01",
        "stop_date": "2025-03-31",
    },
    {
        "idx": 15,
        "item_code": "Reception Desk",
        "qty": 1,
        "rate": "7000",
        "amount": "7000",
        "start_date": "2024-11-10",
        "stop_date": "2024-11-25",
    },
    {
         "idx": 16,
         "item_code": "Breakout Area",
         "qty": 3,
         "rate": "3000",
         "amount": "9000",
         "start_date": "2024-01-01",
         "stop_date": "2024-01-31",
     }
]

target_month = "2024-11"
bill = generate_monthly_bill(item_list, target_month)
print(bill)

{'line_items': [{'item_code': 'Executive Desk (4*2)', 'rate': 1080.0, 'qty': 10, 'amount': 10800.0, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Executive Desk (4*2)', 'rate': 1000.0, 'qty': 5, 'amount': 5000.000000000001, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Manager Cabin', 'rate': 5000.0, 'qty': 5, 'amount': 25000.0, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Parking (2S)', 'rate': 1000.0, 'qty': 10, 'amount': 10000.000000000002, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Parking (2S)', 'rate': 0.0, 'qty': 10, 'amount': 0.0, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Executive Desk (4*2)', 'rate': 1100.0, 'qty': 8, 'amount': 4693.333333333333, 'billing_period': '2024-11-15 to 2024-11-30'}, {'item_code': 'Manager Cabin', 'rate': 5200.0, 'qty': 3, 'amount': 5200.0, 'billing_period': '2024-11-01 to 2024-11-10'}, {'item_code': 'Conference Table', 'rate': 20000.0, 'qty': 1, 'amount': 10666.6